In [1]:
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [2]:
def inf_ing(v_ing):
    inf = v_ing
    if v_ing == 'degorging':
        inf = 'degorge'
    elif v_ing == 'sautéing':
        inf = 'sauté'
    elif v_ing == 'stir-frying':
        inf = 'stir-fry'
    else:
        inf = WordNetLemmatizer().lemmatize(v_ing.lower(), 'v')
    return inf

def infinitive(words):
    aux = []
    word_list = words.split()
    for word in word_list:
        if word != 'cook' and word != 'cooking':
            if word.endswith('ing'):
                inf = inf_ing(word.lower())
                aux.append(inf)
                if inf == word:
                    print('NO INFINITIVE FOR', word)
            else:
                aux.append(word.lower())
    result = ' '.join(aux)
    return result

def trim(s):
    return ' '.join(s.split())

In [3]:
techniques = set()

In [4]:
def clean_1(l):
    r = []
    for e in l:
        if e not in [
            'Cookbook:Cooking Techniques',
            'Cookbook:Barbecue Tips',
            'Cookbook:Beaten Egg',
            'Cookbook:Boiled Pasta',
            'Cookbook:Boiling Rice',
            'Cookbook:Camp Cooking',
            'Cookbook:Cold Water Candy Test',
            'Cookbook:Knife Skills',
            'Cookbook:Mastering the Art of Cooking Paellas',
            'Cookbook:Outdoor Cooking',
            'Cookbook:Paella Cooking Techniques',
            'Cookbook:Roasting Nuts',
            'Cookbook:Secrets to Great Ribs',
            'Cookbook:Separating Eggs',
        ]:
            x = e.strip().lower().replace('_', ' ')
            x = x[len('Cookbook:'):]
            x = x.split(' (')[0]
            x = infinitive(x)
            r.append(x)
    return r

In [5]:
with open('data/techniques/wikibooks_category_cooking_techniques.html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    elems = soup.select('div#mw-pages div.mw-category-group ul li a')
    techs = map(lambda x: trim(x['title']), elems)
    clean_techs = clean_1(techs)
    techniques = techniques.union(clean_techs)

In [6]:
def clean_4(l):
    r = []
    for e in l:
        if e not in [
            'Roasting Nuts',
            'Cold Water Candy Test',
            'Spice rubs',
            'Knife skills',
            'Melting chocolate',
        ]:
            x = e.strip().lower().replace('_', ' ')
            if 'and' in x:
                x1, x2 = x.split(' and ')
                x1 = infinitive(x1)
                x2 = infinitive(x2)
                r.append(x1)
                r.append(x2)
            else:
                x = x.split(' (')[0]
                x = infinitive(x)
                r.append(x)
    return r

In [7]:
with open('data/techniques/wikibooks_cookbook_cooking_techniques.html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    elems = soup.select('div#bodyContent h2 ~ ul:nth-of-type(1) li > a:nth-of-type(1), div#bodyContent h3 ~ ul:nth-of-type(1) li > a:nth-of-type(1)')
    techs = map(lambda x: trim(x.string), elems)
    clean_techs = clean_4(techs)
    techniques = techniques.union(clean_techs)

In [8]:
len(techniques)

71

In [9]:
with open('data/techniques/wikibooks_techniques.txt', 'w') as f:
    f.write('\n'.join(sorted(techniques)))